# Water Usage Model

In [ ]:
import epxexec # Epistemix package for running FRED simulations within Python
from epxexec import epxresults # Expistemix package for managing and accessing FRED simulation outputs
from epxexec.fred_job import fred_job
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

os.environ["FRED_PROJECT"] = str(Path.cwd().parent.parent)

## Household Water Usage Depends on Demographics

The basic water usage model calculates daily household water use. First, the number of individuals (adults at least 16 years old and juveniles younger than 16) in each household is extracted. Then, the household size is used to determine the number of times each household does each activity (except for lawn watering which is done an average of once/week regardless of household size). The following activities are assumed to add to water usage: 

| Activity      | Water used (L) | Frequency |
| ----------- | ----------- |-----------|
| Lawn      | 3000       |  1/week |
| Laundry   | 70        |   1/day |
| Toilet    | 9 | 8/day |
| Shower    | 50 | 1/day |
| Eat/Drink | 8  | 5/day |
| Clean     | 3  | 1/day |


The table above shows the average amount of water used for each activity for a single individual. The total amount of water used each day is drawn from a poisson distribution with the averages shown in the table above and multiplied by household size. 

In [ ]:
water_job = fred_job("main_WATER.fred")
path_to_job=water_job.path_to_job

In [ ]:
path_to_job

In [ ]:
water_dat = pd.read_csv(path_to_job+"/OUT/RUN1/CSV/monthly_household_data.csv")
water_dat['Date'] = pd.to_datetime(water_dat['Date'], format="%Y%m%d")
water_dat

In [ ]:
# single adults
single_adult = water_dat.loc[(water_dat['Num_Adults']==1) & (water_dat['Num_Kids']==0)]
single_adult = single_adult.pivot(index='Date', columns='HH_ID', values='Household_Usage')
mu1 = single_adult.median(axis=1)
lower1 = single_adult.quantile(axis=1, q=.25)
higher1 = single_adult.quantile(axis=1, q=.75)

# multi adult households
multi_adult = water_dat.loc[(water_dat['Num_Adults']>1) & (water_dat['Num_Kids']==0)]
multi_adult = multi_adult.pivot(index='Date', columns='HH_ID', values='Household_Usage')
mu2 = multi_adult.median(axis=1)
lower2 = multi_adult.quantile(axis=1, q=.25)
higher2 = multi_adult.quantile(axis=1, q=.75)

# households with children
with_kids = water_dat.loc[(water_dat['Num_Adults']>0) & (water_dat['Num_Kids']>0)]
with_kids = with_kids.pivot(index='Date', columns='HH_ID', values='Household_Usage')
mu3 = with_kids.median(axis=1)
lower3 = with_kids.quantile(axis=1, q=.25)
higher3 = with_kids.quantile(axis=1, q=.75)

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(mu1, lw=2, label='Single Adult', color='blue')
ax.plot(mu2, lw=2, label='Multi Adult', color='purple')
ax.plot(mu3, lw=2, label='Adults with Kids', color='green')
ax.legend(loc='best')
fig.autofmt_xdate()
ax.set_xlabel('Date')
ax.set_ylabel('Monthly Water Usage')
ax.set_ylim(10000, 40000)
ax.grid()